In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

C:\Users\344929\Anaconda2\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\344929\Anaconda2\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\344929\Anaconda2\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\344929\Anaconda2\envs\keras\lib\site-packages\tensorflow\pyth

In [2]:
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import argparse
import pickle

In [3]:
NUM_EPOCHS = 10
INIT_LR = 1e-2
BS = 32

In [4]:
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

In [5]:
trainX.shape

(60000, 28, 28)

In [6]:
trainY.shape

(60000,)

In [7]:
from matplotlib import pyplot
from matplotlib.image import imread
pyplot.imshow(trainX[9])
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [8]:
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [9]:
trainX = trainX[:10000]
trainY = trainY[:10000]

In [10]:
trainX.shape
trainY.shape

(10000,)

In [11]:
testX = testX[:2000]
testY = testY[:2000]

In [12]:
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

In [13]:
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

In [14]:
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

In [15]:
import logging
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score
from tensorflow.python.keras.callbacks import Callback
import numpy as np

# Validation metrics callback: validation precision, recall and F1
# Some of the code was adapted from https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2
class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y if (y.ndim == 1 or y.shape[1] == 1) else np.argmax(y, axis=1)        
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []        
    
    def on_epoch_end(self, epoch, logs={}):                
        x_test = self.x
        y_test = self.y
        val_targ = y_test
        predictions = self.model.predict(x_test)
        val_predict = predictions.argmax(axis=1)        
        val_f1 = round(f1_score(val_targ, val_predict,average='micro'), 4)
        val_recall = round(recall_score(val_targ, val_predict,average='micro'), 4)
        val_precis = round(precision_score(val_targ, val_predict,average='micro'), 4)        
        
        self.val_f1s.append(val_f1)
        self.val_recalls.append(val_recall)
        self.val_precisions.append(val_precis)
        
        
        # Add custom metrics to the logs, so that we can use them with
        # EarlyStop and csvLogger callbacks
        logs["val_f1"] = val_f1
        logs["val_recall"] = val_recall
        logs["val_precis"] = val_precis

        print("— val_f1: {} — val_precis: {} — val_recall {} ".format(
                 val_f1, val_precis, val_recall))
                        
        return    

In [16]:
metrics = Metrics(testX,testY)
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model = MiniVGGNet.build(width=28, height=28, depth=1, classes=10)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
H = model.fit(trainX, trainY,validation_data=(testX, testY), batch_size=BS, epochs=NUM_EPOCHS, verbose=1, callbacks=[metrics])

Train on 10000 samples, validate on 2000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
10000/10000 [==============================] - 50s 5ms/sample - loss: 0.8695 - acc: 0.7279 - val_loss: 0.6786 - val_acc: 0.7500
Epoch 2/10
  544/10000 [>.............................] - ETA: 37s - loss: 0.5601 - acc: 0.8070

In [ ]:
print(H.params.keys())

In [ ]:
print(metrics.val_f1s)

In [ ]:
print(metrics.val_precisions)

In [ ]:
print(metrics.val_recalls)

In [ ]:
pyplot.plot(H.history['acc'])
pyplot.plot(H.history['val_acc'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.show()

In [ ]:
preds = model.predict(testX)

In [ ]:
testY.argmax(axis=1)

In [ ]:
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))